## SDKを用いた計装

必要なライブラリをダウンロードします。

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


実行に必要な環境変数を読み込みます。

In [6]:
import os, warnings, time
from dotenv import load_dotenv

load_dotenv("../.env")

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://localhost:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

'pk-1234567890'

LangfuseのSDKクライアントを初期化します。

In [7]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

### Decorator

`@obserbe()`デコレーターを用いてTraceと各種Observation(Span/Generation)の確認をします。

In [8]:
from langfuse.decorators import observe, langfuse_context

@observe()
def meet_the_bigbaby():
    print("Meet the BigBaBy!")

@observe(name="Search the BigBaBy's dictionary")
def search_the_bigbaby_dict():
    time.sleep(2)

@observe(name="Ask the BigBaBy", as_type="generation")
def ask_the_bigbaby(query: str):
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        model = "gpt-4o-mini"
        llm = ChatOpenAI(api_key=openai_api_key, model=model)
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        model = "command-r-plus"
        llm = ChatCohere(cohere_api_key=cohere_api_key, model=model)
    input = [
        {"role": "user", "content": query}
    ]
    result = llm.invoke(input=input)
    return result.content

@observe(name="Ask the BigBaby")
def main():
    meet_the_bigbaby()
    search_the_bigbaby_dict()
    return ask_the_bigbaby(query="肉料理のおすすめ教えて")

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)
response = main()
print(f"{response=}")

Meet the BigBaBy!
validate_env
generate
{'message': '肉料理のおすすめ教えて', 'chat_history': [], 'model': 'command-r-plus'}
basecohere.chat
text='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンで焼いた伝統的なイギリス料理です。焼き加減を調節して、好みの仕上がりにしたものを薄切りにして食べます。\n- プルコギ： 韓国の伝統的な料理で、薄切りにした牛肉や野菜を甘辛いタレで炒めたものです。ご飯と一緒に食べるのが一般的で、ごま油の風味とタレの甘辛さが食欲をそそります。\n- 酢豚： 中華料理の定番で、豚肉と野菜を甘酸っぱいタレで炒めた料理です。サクサクとした衣の食感と、酸味と甘みのバランスが絶妙です。\n- チキンティッカ： インド料理の代表的なグリル料理で、ヨーグルトと香辛料に漬け込んだ鶏肉をグリルしたものです。香辛料の香りとヨーグルトのまろやかさが特徴です。\n- ラムチョップ： 子羊の骨付き肉をグリルまたはローストしたもので、柔らかくジューシーな食感が特徴です。香草やスパイスで味付けして食べるのが一般的です。\n\nこれらの料理は世界中で愛されている肉料理の一部に過ぎませんが、ぜひお試しいただき、お好みの料理を見つけてください。' generation_id='5137b917-2166-4225-86f1-454129f58c2a' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_User(message='肉料理のおすすめ教えて', tool_calls=None, role='USER'), Message_Chatbot(message='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンで焼いた伝統的なイギリス料理です。焼き加減を

### Low-Level SDK

#### 属性情報を付与しない実行例

Low-Level SDKを用いてTraceと各種Observation(Event/Span/Generation)の確認をします。

In [9]:
# Traceを作成する
trace = langfuse.trace(
    name="Ask the BigBaBy"
)

# Observation - EventをTraceに追加する
event = trace.event(
    name="Meet the BigBaBy"
)

# Observation - SpanをTraceに追加する
span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 2秒間スリープ（同期コールする検索システムを想定）
time.sleep(2)
span.end()

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    llm = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    llm = ChatCohere(cohere_api_key=cohere_api_key, model=model)

input = [
    {"role": "user", "content": "おすすめの肉料理おしえて"}
]

generation = trace.generation(
    name="Ask the BigBaBy",
    model=model,
    input=input
)

result = llm.invoke(input=input)

generation.end(
    output=result.content,
)

validate_env
generate
{'message': 'おすすめの肉料理おしえて', 'chat_history': [], 'model': 'command-r-plus'}
basecohere.chat
text='おすすめの肉料理はたくさんありますが、ここでは3つの料理を紹介します。\n\n1. 牛丼\n牛丼は、柔らかく煮た牛肉と甘辛いタレ、玉ねぎをご飯の上にのせた日本の伝統的な料理です。シンプルな料理ですが、牛肉の旨味とタレの甘辛さが相まって、食欲をそそる一品です。\n\n2. ステーキ\nステーキは、牛肉の塊を焼いた料理で、世界中で愛されている肉料理の定番です。焼き加減やカットによってさまざまな味わいを楽しむことができ、付け合わせの野菜やソースによっても味に変化をつけることができます。\n\n3. 酢豚\n酢豚は、中華料理の代表的な肉料理の一つで、甘酸っぱい味付けが特徴です。豚肉に片栗粉をまぶして油で揚げ、野菜と共に甘酸っぱいタレで炒め合わせます。ご飯ともよく合うので、メインのおかずとしておすすめです。\n\n以上、3つの肉料理を紹介しました。ぜひお試しください。' generation_id='aa0841c7-3ee7-4cbb-b2e3-8dcace7672e5' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_User(message='おすすめの肉料理おしえて', tool_calls=None, role='USER'), Message_Chatbot(message='おすすめの肉料理はたくさんありますが、ここでは3つの料理を紹介します。\n\n1. 牛丼\n牛丼は、柔らかく煮た牛肉と甘辛いタレ、玉ねぎをご飯の上にのせた日本の伝統的な料理です。シンプルな料理ですが、牛肉の旨味とタレの甘辛さが相まって、食欲をそそる一品です。\n\n2. ステーキ\nステーキは、牛肉の塊を焼いた料理で、世界中で愛されている肉料理の定番です。焼き加減やカットによってさまざまな味わいを

#### 属性情報を付与した実行例

属性情報などを追加した版です。

In [20]:
# Traceを作成する
trace = langfuse.trace(
    name="Ask the BigBaBy"
)

trace_id = trace.id

# Observation - EventをTraceに追加する
event = trace.event(
    name="Meet the BigBaBy"
)

# Observation - SpanをTraceに追加する
span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 2秒間スリープ（同期コールする検索システムを想定）
time.sleep(2)
# ダミーの検索結果
search_result = {
    "recipes": [{
        "roast-beaf": "ローストビーフ： 牛肉の塊をオーブンで焼いた料理で..."
    }]
}
span.end()
span.update(
    input="おすすめの肉料理おしえて",
    output=search_result
)

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    llm = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    llm = ChatCohere(cohere_api_key=cohere_api_key, model=model)

input = [
    {"role": "user", "content": "おすすめの肉料理おしえて"}
]

generation = trace.generation(
    name="Ask the BigBaBy",
    model=model,
    input=input
)

result = llm.invoke(input=input)

generation.end(
    output=result.content,
)

trace.update(
    input="おすすめの肉料理おしえて",
    output=result.content
)

validate_env
invoke
generate
{'message': 'おすすめの肉料理おしえて', 'chat_history': [], 'model': 'command-r-plus'}
basecohere.chat
text='おすすめの肉料理はたくさんありますが、ここでは3つの料理を紹介します。\n\n1. ローストビーフ： 牛肉の塊をオーブンで低温でゆっくりと焼き上げた料理です。中がピンク色に焼き上がったローストビーフは、やわらかくジューシーで、赤ワインやホースラディッシュソースなどと一緒に楽しめます。\n\n2. 豚の角煮： 豚バラ肉を醤油、砂糖、酒、みりんなどで長時間煮込んだ料理です。とてもやわらかく、ホロホロとした食感が楽しめます。ご飯にもお酒にもよく合う一品です。\n\n3. チキンティッカ： インド料理のグリル料理で、ヨーグルトと香辛料に漬け込んだ鶏肉を串に刺して焼いたものです。香り豊かでジューシーな味わいが特徴です。カレー粉やターメリックなどのスパイスが効いているので、ビールやワインなどのお酒にもよく合います。\n\nぜひお試しください！' generation_id='499115d3-615a-412d-a2e9-a58ad8f9a898' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[Message_User(message='おすすめの肉料理おしえて', tool_calls=None, role='USER'), Message_Chatbot(message='おすすめの肉料理はたくさんありますが、ここでは3つの料理を紹介します。\n\n1. ローストビーフ： 牛肉の塊をオーブンで低温でゆっくりと焼き上げた料理です。中がピンク色に焼き上がったローストビーフは、やわらかくジューシーで、赤ワインやホースラディッシュソースなどと一緒に楽しめます。\n\n2. 豚の角煮： 豚バラ肉を醤油、砂糖、酒、みりんなどで長時間煮込んだ料理です。とてもやわらかく、ホロホロとした食感が楽し